# MOVIE LENS 10M

In [92]:
import pandas as pd
import numpy as np
import json

In [93]:
def load_ratings():

    return pd.read_csv(
        './dataset/ratings.dat',
        sep='::',
        header=None,
        names=['userId', 'movieId', 'rating', 'timestamp'])
      
def load_movies():

    return pd.read_csv(
        './dataset/movies.dat',
        sep='::',
        header=None,
        names=['movieID', 'Title', 'Genre'])


def get_movies_id_list():

        movies = pd.read_csv(
        './dataset/movies.dat',
        sep='::',
        header=None,
        names=['movieID', 'Title', 'Genre'])

        retorno = movies.values[:, 0]
        return retorno


def get_windows(n_items, n_windows=40):
    """
    Returns the items partitions made.
    Both first item and last item are included.

    Returns:
        List of tuples, where the tuples are 
        (first_item_window_INCLUDED, last_item_window_INCLUDED)
    """

    Q, q = divmod(n_items, n_windows)
    index = 0
    windows = []

    for j in range(n_windows):

        if j < q: #Cambiado de su paper porque no sense el =
          n_items_window = Q + 1

        else:
          n_items_window = Q

        windows.append((index, index + n_items_window - 1))
        index = index + n_items_window 

    return windows
    

def obtain_vectors(J=40):

    data = load_ratings()

    users_ids = list(set(data['userId']))
    n_users = len(users_ids)
    movies_ids = get_movies_id_list()
    n_movies = len(movies_ids)

    windows = get_windows(n_movies, J)
    vectors = { **dict.fromkeys([i for i in users_ids], [])} 

    for user_id in users_ids:

        ratings_user = data.loc[data["userId"] == user_id].values
        n_ratings_user = len(ratings_user)

        user_vector = []

        for window in windows:

            movies_window = movies_ids[window[0] : window[1]]
            ratings_user_window = [rating for rating in ratings_user if int(rating[1]) in movies_window]
            n_ratings_user_window = len(ratings_user_window)

            user_vector.append(n_ratings_user_window) #NRW
            user_vector.append(n_ratings_user_window/n_ratings_user) #NRWR

        vectors[user_id] = user_vector


    with open('vectors-genuine-users-ML10M.json', 'w') as f:
        json_dumps_str = json.dumps(vectors, indent=2)
        print(json_dumps_str, file=f)
        f.close()

    return vectors

In [94]:
print(obtain_vectors())

{1: [3, 0.13636363636363635, 13, 0.5909090909090909, 6, 0.2727272727272727, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0], 2: [3, 0.15, 1, 0.05, 9, 0.45, 3, 0.15, 2, 0.1, 2, 0.1, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0], 3: [3, 0.09090909090909091, 0, 0.0, 1, 0.030303030303030304, 0, 0.0, 5, 0.15151515151515152, 4, 0.12121212121212122, 1, 0.030303030303030304, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 1, 0.030303030303030304, 1, 0.030303030303030304, 0, 0.0, 0, 0.0, 1, 0.030303030303030304, 1, 0.030303030303030304, 

C:\Users\patri\AppData\Local\Temp\ipykernel_12720\2859184455.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(
C:\Users\patri\AppData\Local\Temp\ipykernel_12720\2859184455.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(
